# Simulation Based Inference

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

## Simulation configuration and priors

We start with configuring a TVB simulator instance which will serve as a template for the inference, here we take the same setup as in the previous notebooks.

In [ ]:
from tvb.simulator.lab import *

In [ ]:
from tvb_inversion.utils import init_experiment, data_path
from tvb_inversion.parameters import SimSeq

In [ ]:
seq = SimSeq(
    template = simulator.Simulator(
        model=mpr,
        connectivity=conn,
        coupling=coupling.Linear(),
        integrator=integrators.HeunStochastic(
            dt=0.01,
            noise=noise.Additive(nsig=0.037*np.r_[1,2])
        ),
        monitors=[monitors.TemporalAverage(period=.1)]                
    ).configure(),
    params=['coupling.a'],
    values=[
        [
            np.r_[a],                           # coupling scaling G
        ] 
        for a in np.arange(0.5, 1.2,0.05)       # list of values of G
    ]
)

Next, we define the prior distribution for the parameters we want to subject to inference using the `Prior` class from the `tvb-inversion` package. Here a two-dimensional uniform prior is given for the coupling strength parameter, and the $\eta_{limbic}$ value:

In [ ]:
from tvb_inversion.sbi import Prior
import torch

prior = Prior(['coupling.a'], torch.distributions.Uniform(0.1, 1.2))

## Sampling, running simulations

Here we follow the same steps as in the case of parameter sweeps. We could run the simulations locally using following the local parallel runner as shown below, or use the distributed runner as in the previous examples.

```python
import utils

utils.run_local(seq, metrics, backend=NbMPRBackend, checkpoint_dir='test_run', filename='results.npy')
```


## Inference, diagnostics

Having finished the simulations, we can continue by training the estimator and constructing the posterior. 

In [ ]:
from tvb_inversion.sbi import EstimatorSBI

estimator = EstimatorSBI(prior, seq=seq, metrics=metrics)
summ_stats = estimator.load_summary_stats('results.npy')

Next we train the estimator on the summary statistics of the simulated data features.

In [ ]:
posterior = estimator.train(summ_stats)

The trained estimator now can in turn be used to sample from the posterior given the summary statistics of the empirical data:

In [ ]:
num_samples = 20_000
posterior_samples = posterior.sample((num_samples,), obs_stats)

And finally, to assess the result, we can compute the posterior shrinkage -- a diagnostic value saying how much the empirical data is able to inform the value of the parameters of interest. Ideally, this value is close to 1 indicating a well identified posterior:

In [ ]:
from tvb_inversion.sbi.inference import shrinkage, zscore

post_std = torch.std(posterior_samples)
post_mean = torch.mean(posterior_samples)

shr = shrinkage(prior_std, post_std)